In [1]:
import numpy as np
import pandas as pd
import neuroseries as nts
import scipy.io

import pickle

import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 

import basefunction.assembly as assembly

import bk.load
import bk.plot
import bk.signal
import bk.compute

import basefunction.vBaseFunctions3 as vbf

import basefunction.anass_load as at

In [2]:
path = '/mnt/electrophy/Facundo/Behavior/Rat37/Rat37-010721/Rat37-f_210701_105158/'

files = 'Basler_acA1300-200uc__23135826__20210701_105207962DLC_resnet50_LinearTrackMay25shuffle1_1030000_filtered.h5'
meta = 'Basler_acA1300-200uc__23135826__20210701_105207962DLC_resnet50_LinearTrackMay25shuffle1_1030000_meta.pickle'

In [129]:
path_meta = os.path.join(path,meta)
path_pos = os.path.join(path,files)

pos = pd.read_hdf(path_pos)

with open(path_meta,'rb') as f:
    data = pickle.load(f)
    x1,x2,y1,y2 = data['data']['cropping_parameters']

for p in pos:
    if p[2].lower() == 'x': pos[p] = pos[p]+x1
    if p[2].lower() == 'y': pos[p] = pos[p]+y1

    

In [130]:
x1,x2

(234, 1146)

In [127]:
pos = pos[pos.columns[0][0]]

In [128]:
plt.plot(pos['snout']['x'],pos['snout']['y'])

In [112]:
for p in pos:
    if p[2].lower() == 'x': pos[p] = pos[p]+x1
    if p[2].lower() == 'y': pos[p] = pos[p]+y1

In [111]:
pos

scorer    DLC_resnet50_LinearTrackMay25shuffle1_1030000                        \
bodyparts                                         snout                         
coords                                                x          y likelihood   
0                                            264.607674  64.251938   0.999989   
1                                            264.758812  64.307510   0.999993   
2                                            264.945656  64.608002   0.999997   
3                                            264.945656  64.608002   0.999997   
4                                            264.945656  64.608002   0.999996   
...                                                 ...        ...        ...   
26635                                        231.306723  74.393318   0.670303   
26636                                        233.666311  79.709335   0.727336   
26637                                        234.954576  80.045982   0.887712   
26638                                        234.954576  80.045982   0.981521   
26639                                        234.954576  79.709335   0.999034   

scorer                                                                         \
bodyparts        neck                            m_body                         
coords              x          y likelihood           x          y likelihood   
0          277.451344  50.303513   1.000000  301.864342  44.189457        1.0   
1          277.468323  50.536316   1.000000  301.873856  44.189457        1.0   
2          277.468323  50.542454   1.000000  301.873856  44.189457        1.0   
3          277.451344  50.542454   1.000000  301.909340  44.158306        1.0   
4          277.468323  50.542454   1.000000  301.892189  44.157932        1.0   
...               ...        ...        ...         ...        ...        ...   
26635      256.610481  64.750008   0.999998  289.556568  58.460762        1.0   
26636      256.465582  64.734528   0.999999  288.926041  57.957947        1.0   
26637      256.465582  64.734528   0.999999  288.872208  57.619492        1.0   
26638      256.374020  64.734528   0.999999  288.717968  56.146286        1.0   
26639      256.374020  64.734528   0.999999  288.565422  55.502602        1.0   

scorer                 ...                                               \
bodyparts      e_body  ...                  l_ear                         
coords              x  ... likelihood           x          y likelihood   
0          334.000069  ...        1.0  274.952301  59.381981   1.000000   
1          334.000069  ...        1.0  274.954548  59.688457   1.000000   
2          334.000069  ...        1.0  274.954548  59.688457   1.000000   
3          333.913620  ...        1.0  274.954548  59.647610   1.000000   
4          333.913620  ...        1.0  274.897205  59.601059   1.000000   
...               ...  ...        ...         ...        ...        ...   
26635      318.270760  ...        1.0  249.572491  74.096115   0.999995   
26636      318.270760  ...        1.0  249.572491  74.474739   0.999992   
26637      318.189644  ...        1.0  250.754208  75.165436   0.999998   
26638      318.124008  ...        1.0  250.754208  75.165436   1.000000   
26639      317.956512  ...        1.0  250.754208  75.165436   1.000000   

scorer                                                                         
bodyparts       r_ear                            e_tail                        
coords              x          y likelihood           x          y likelihood  
0          268.056870  53.103573   1.000000  397.498337  68.463364        1.0  
1          268.165192  53.178490   1.000000  397.753845  68.481873        1.0  
2          268.165192  53.432198   1.000000  397.828415  68.487679        1.0  
3          268.136852  53.382301   1.000000  397.870743  68.545624        1.0  
4          268.165192  53.382301   1.000000  398.181519  68.545624        1.0  
...               ...        ...       

In [39]:
b = pd.concat((a,a))

In [69]:
c = b.reset_index(drop = True)

In [77]:
c['DLC_resnet50_LinearTrackMay25shuffle1_1030000']['l_ear']

coords,x,y,likelihood
0,40.952301,59.381981,1.000000
1,40.954548,59.688457,1.000000
2,40.954548,59.688457,1.000000
3,40.954548,59.647610,1.000000
4,40.897205,59.601059,1.000000
...,...,...,...
53275,15.572491,74.096115,0.999995
53276,15.572491,74.474739,0.999992
53277,16.754208,75.165436,0.999998
53278,16.754208,75.165436,1.000000


In [41]:
for a in b: print(a)

('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'snout', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'snout', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'snout', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'neck', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'neck', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'neck', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'm_body', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'm_body', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'm_body', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'e_body', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'e_body', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'e_body', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'l_ear', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'l_ear', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'l_ear', 'likel

In [33]:
for a in b: print(a)

('index', '', '')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'snout', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'snout', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'snout', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'neck', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'neck', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'neck', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'm_body', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'm_body', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'm_body', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'e_body', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'e_body', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'e_body', 'likelihood')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'l_ear', 'x')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000', 'l_ear', 'y')
('DLC_resnet50_LinearTrackMay25shuffle1_1030000

In [73]:
def states():
    #BK : 17/09/2020
    #Return a dict with variable from States.
#     if session_path == 0 : session_path = get_session_path(session_name)
    states = scipy.io.loadmat(path + '/States.mat')
    
    useless  = ['__header__','__version__','__globals__']
    for u in useless:
        del states[u]
    states_ = {}
    for state in states:
        states_.update({state:nts.IntervalSet(states[state][:,0],states[state][:,1],time_units = 's')})
    
    return states_

def notScored(states,start,stop):

    """ 
    States : Dict with variable from States.
    Start : Start ot the recording
    Stop : End of the recording (to know until when it should be scored)
    """
    fullRec = nts.IntervalSet(start,stop,time_units= 's')
    all_intervals = nts.IntervalSet(0,0)

    for s in states:
        all_intervals = all_intervals.union(states[s])

    all_intervals = all_intervals.drop_short_intervals(5)
    notLabeled = fullRec.set_diff(all_intervals)
    return notLabeled

In [71]:
path = '/mnt/electrophy/IcyBox/Rat08-20130713/'


In [74]:
states = states()

In [80]:
for s in states:
    print(states[s].as_units('s'))

      start      end
0    1927.0   2013.0
1    2430.0   2603.0
2    3276.0   3371.0
3    3842.0   3887.0
4    3917.0   3969.0
5    3993.0   4023.0
6    4530.0   4567.0
7    4677.0   4838.0
8    4864.0   4920.0
9    5427.0   5488.0
10   6070.0   6206.0
11   7312.0   7374.0
12   7760.0   7787.0
13   7910.0   8030.0
14   8798.0   8852.0
15   9334.0   9402.0
16  16146.0  16405.0
17  16766.0  16787.0
18  17260.0  17363.0
19  17430.0  17496.0
20  17811.0  17851.0
21  18077.0  18131.0
22  18529.0  18614.0
23  18980.0  19084.0
24  19168.0  19186.0
25  19449.0  19545.0
26  19772.0  19817.0
27  21287.0  21358.0
28  21630.0  21674.0
29  21851.0  21953.0
      start      end
0    1448.0   1697.0
1    1777.0   1926.0
2    2104.0   2429.0
3    2826.0   3275.0
4    3383.0   3841.0
5    3888.0   3916.0
6    3970.0   3992.0
7    4024.0   4529.0
8    4568.0   4648.0
9    5044.0   5426.0
10   5489.0   6069.0
11   6368.0   6629.0
12   6970.0   7311.0
13   7375.0   7759.0
14   7788.0   7909.0
15   8485.0  

In [9]:
all_intervals = nts.IntervalSet(0,0)


In [70]:
notScored(states,0,30000).drop_short_intervals(1,time_units='s').as_units('s')

,start,end
0,0.0,5.0
1,7606.0,7631.0
2,8084.0,8209.0
3,17878.0,30000.0


In [67]:
def wavelet_spectrogram(lfp, fmin, fmax, nfreq):
    f_wv = pow(2, np.linspace(np.log2(fmin), np.log2(fmax), nfreq))
    t = lfp.index.values
    output = vbf.wvSpect(lfp.values, f_wv)

    return f_wv, t, output[0]

In [3]:
bk.load.current_session_linux()

In [ ]:
states = bk.load.states()
stru = ['Hpc','BLA']

In [ ]:
lfp = {}
for s in stru: lfp.update({s:bk.load.lfp_in_intervals(bk.load.random_channel(s),states['Rem'])})

In [19]:
lfp_filt = {}
for s in stru: lfp_filt.update({s:bk.signal.passband(lfp[s],4,10)})

In [21]:
fig,ax = plt.subplots(3,1,sharex=True,sharey=True)

for a,s in zip(ax,stru):
    a.plot(lfp[s],color = 'Grey')
    a.plot(lfp_filt[s],color = 'Red')
col = ['Blue','Green']
for s,c in zip(stru,col):
    ax[2].plot(lfp_filt[s],color = c)

/home/billel/anaconda3/lib/python3.8/site-packages/ipykernel/eventloops.py:105: MatplotlibDeprecationWarning: Toggling axes navigation from the keyboard is deprecated since 3.3 and will be removed two minor releases later.
  app.exec_()


In [22]:
lfp_hpc_sws = bk.load.lfp_in_intervals(23,states['sws'])

In [23]:
ripples = bk.load.ripples()

In [70]:
spec = []
for t_rip in ripples['peak'].as_units('s').index.values[0:15]:
    inter = nts.IntervalSet(t_rip-0.10,t_rip+0.10,time_units='s')
    l = lfp_hpc_sws.restrict(inter)
    
    f, t, Sxx = wavelet_spectrogram(l,30,500,50)
    spec.append(Sxx)

In [65]:
for i,s in enumerate(spec):
    if len(s) == 249: spec[i] = np.append(spec[i],0)

In [69]:
fig,ax = plt.subplots(211)
ax[0].pcolormesh(t, f_wv, wave, shading="gouraud", vmin=20, vmax=350, rasterized=True)


ValueError: not enough values to unpack (expected 2, got 1)

In [3]:
import numpy as np
import pandas as pd
import neuroseries as nts
import scipy.io

import pickle

import time
import matplotlib.pyplot as plt
%matplotlib qt

import seaborn as sns 

import basefunction.assembly as assembly

import bk.load
import bk.plot
import bk.signal
import bk.compute

import basefunction.vBaseFunctions3 as vbf

import basefunction.anass_load as at

In [4]:
def main(path = 'Rat08/Rat08-20130713',args = None):
    output_path = args[0]
    
    bk.load.current_session_linux(
        base_folder='/media/billel/GGLab-01-BK/DATA/GG-Dataset',
        local_path = path)
    
    states = bk.load.states()
    lfp = {}
    channels = {
        'hpc'  :bk.load.ripple_channel(),
        'bla'  :bk.load.bla_channels(),
        'accel':bk.load.channels()[22][0]}

    lfp.update({'hpc':bk.load.lfp(channels['hpc'],memmap=True)})
    lfp.update({'bla_left':bk.load.lfp(channels['bla']['left'],memmap=True)})
    lfp.update({'bla_right':bk.load.lfp(channels['bla']['right'],memmap=True)})
    coords = ['x','y','z']

    for chan,coord in zip(channels['accel'],coords):
        lfp.update({f'accel{coord}':bk.load.lfp(chan,memmap=True)})

    print(lfp['hpc'].shape)
    np.savez_compressed(f'{output_path}/{bk.load.session}',
                        hpc = lfp['hpc'],
                        bla_left = lfp['bla_left'],
                        bla_right = lfp['bla_right'],
                        accelx = lfp['accelx'],
                        accely = lfp['accely'],
                        accelz = lfp['accelz'],
                        states = states)

In [5]:
bk.load.batch(main,['/home/billel/pCloudDrive/IFM/Data/Saoud/'],linux = True)

100%|██████████| 61/61 [2:05:20<00:00, 123.29s/it]

Batch finished in 7520.748377561569
Some session were not processed correctly
['Rat08-20130708', 'Rat08-20130709', 'Rat08-20130710', 'Rat08-20130711', 'Rat08-20130718', 'Rat08-20130719', 'Rat08-20130720', 'Rat08-20130722', 'Rat09-20140324', 'Rat09-20140325', 'Rat09-20140326', 'Rat09-20140327', 'Rat09-20140328', 'Rat09-20140329', 'Rat09-20140331', 'Rat09-20140401', 'Rat09-20140402', 'Rat09-20140403', 'Rat09-20140404', 'Rat09-20140405', 'Rat09-20140407', 'Rat09-20140408', 'Rat09-20140409', 'Rat10-20140703', 'Rat10-20140704', 'Rat10-20140707', 'Rat11-20150331', 'Rat11-20150401', 'Rat11-20150402', 'Rat11-20150403']
49.18032786885246 %


{'Rat08-20130712': None,
 'Rat08-20130713': None,
 'Rat08-20130715': None,
 'Rat08-20130716': None,
 'Rat08-20130717': None,
 'Rat10-20140619': None,
 'Rat10-20140620': None,
 'Rat10-20140622': None,
 'Rat10-20140624': None,
 'Rat10-20140626': None,
 'Rat10-20140627': None,
 'Rat10-20140628': None,
 'Rat10-20140629': None,
 'Rat10-20140701': None,
 'Rat10-20140702': None,
 'Rat10-20140705': None,
 'Rat10-20140708': None,
 'Rat11-20150310': None,
 'Rat11-20150312': None,
 'Rat11-20150313': None,
 'Rat11-20150314': None,
 'Rat11-20150315': None,
 'Rat11-20150316': None,
 'Rat11-20150317': None,
 'Rat11-20150321': None,
 'Rat11-20150323': None,
 'Rat11-20150325': None,
 'Rat11-20150326': None,
 'Rat11-20150327': None,
 'Rat11-20150328': None,
 'Rat11-20150330': None}

In [3]:
main(args=['/home/billel/pCloudDrive/IFM/Data/Saoud/'])

Rat : 8 on day : 6
Working with session Rat08-20130713 @ /media/billel/GGLab-01-BK/DATA/GG-Dataset/Rat08/Rat08-20130713
(28350000,)
